# 2. Ingeniería de características I
## 2.1 Datos demográficos.



In [1]:
import pandas as pd
from pathlib import Path

# 1. Definir la ruta a los datos intermedios
# Según tu configuración, INTERIM_DATA_DIR es data/1_interim/
interim_path = Path("/workspace/TFM_education_ai_analytics/data/1_interim")

# 2. Cargar los DataFrames
print("Cargando datos desde interim...")
df_students = pd.read_csv(interim_path / "students.csv")
df_assessments = pd.read_csv(interim_path / "assessments.csv")
df_interactions = pd.read_csv(interim_path / "interactions.csv")

# 3. Crear el ID único (necesario para el Feature Engineering posterior)
# Este ID combina Estudiante + Curso + Semestre
for df in [df_students, df_assessments, df_interactions]:
    df['unique_id'] = (df['id_student'].astype(str) + '_' + 
                      df['code_module'] + '_' + 
                      df['code_presentation'])

# 4. Mostrar información básica
print(f"\n📊 Resumen de carga:")
print(f"- Estudiantes únicos (en intentos de curso): {df_students['unique_id'].nunique()}")
print(f"- Total de evaluaciones registradas: {len(df_assessments)}")
print(f"- Total de interacciones en el VLE: {len(df_interactions)}")

# Ver una muestra de los estudiantes
df_students.head()

Cargando datos desde interim...

📊 Resumen de carga:
- Estudiantes únicos (en intentos de curso): 32593
- Total de evaluaciones registradas: 173912
- Total de interacciones en el VLE: 10655280


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,unique_id
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268,11391_AAA_2013J
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268,28400_AAA_2013J
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,-92.0,12.0,268,30268_AAA_2013J
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268,31604_AAA_2013J
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268,32885_AAA_2013J


In [2]:
from sklearn.model_selection import train_test_split

# 1. Obtener la lista de estudiantes únicos y su resultado para estratificar
# (Usamos el primer resultado que aparezca para cada id_student)
student_labels = df_students.groupby('id_student')['final_result'].first()
unique_students = student_labels.index
labels = student_labels.values

# 2. Dividir: 70% Train, 30% Temporal (que luego dividiremos en Val y Test)
train_ids, temp_ids, y_train, y_temp = train_test_split(
    unique_students, 
    labels,
    test_size=0.3, 
    random_state=42,
    stratify=labels
)

# 3. Dividir el Temporal en 50% Validación y 50% Test (esto da 15% y 15% del total)
val_ids, test_ids, y_val, y_test = train_test_split(
    temp_ids, 
    y_temp,
    test_size=0.5, 
    random_state=42,
    stratify=y_temp
)

print(f"Total estudiantes: {len(unique_students)}")
print(f"Train: {len(train_ids)} | Val: {len(val_ids)} | Test: {len(test_ids)}")

Total estudiantes: 28785
Train: 20149 | Val: 4318 | Test: 4318


In [3]:
print(f"Número de matriculaciones (filas): {len(df_students)}")
print(f"Número de personas (IDs únicos): {df_students['id_student'].nunique()}")

Número de matriculaciones (filas): 32593
Número de personas (IDs únicos): 28785


In [4]:
# 1. Definir una función para filtrar cualquier DataFrame por una lista de IDs de estudiante
def filter_by_ids(df, ids):
    return df[df['id_student'].isin(ids)].copy()

# 2. Crear los subconjuntos (Splits) de entrenamiento
print("Creando subconjuntos de datos...")
train_students = filter_by_ids(df_students, train_ids)
train_assessments = filter_by_ids(df_assessments, train_ids)
train_interactions = filter_by_ids(df_interactions, train_ids)

# 3. Crear los subconjuntos de validación (para evaluar luego)
val_students = filter_by_ids(df_students, val_ids)
val_assessments = filter_by_ids(df_assessments, val_ids)
val_interactions = filter_by_ids(df_interactions, val_ids)

# 4. Crear los subconjuntos de test (Evaluación final)
test_students = filter_by_ids(df_students, test_ids)

print(f"✅ OK: train_students definido con {len(train_students)} matrtículas.")

Creando subconjuntos de datos...
✅ OK: train_students definido con 22785 matrtículas.


In [16]:
import re
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 1. CONSTANTES Y COMPONENTES (Todo lo necesario en una sola celda)
imd_map = {
    '0-10%': 0, '10-20%': 1, '20-30%': 2, '30-40%': 3, '40-50%': 4,
    '50-60%': 5, '60-70%': 6, '70-80%': 7, '80-90%': 8, '90-100%': 9
}
age_map = {'0-35': 0, '35-55': 1, '55<=': 2}

# Componentes de Scikit-Learn
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Configuración de columnas
cat_cols = ['gender', 'region', 'highest_education', 'disability']
cols_to_scale = ['imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits']

def clean_column_names(names):
    return [re.sub(r'[^a-zA-Z0-9_]', '_', name.replace(' ', '_')).lower() for name in names]

def process_demographics_raw(df_input):
    df = df_input.copy()
    
    # Solo imputación y mapeo (sin escalar)
    moda_age = df['age_band'].mode()[0] if not df['age_band'].mode().empty else '0-35'
    moda_imd = df['imd_band'].mode()[0] if not df['imd_band'].mode().empty else '50-60%'
    
    df['imd_band'] = df['imd_band'].fillna(moda_imd).map(imd_map).fillna(5).astype(int)
    df['age_band'] = df['age_band'].fillna(moda_age).map(age_map).fillna(0).astype(int)
    
    # One-Hot Encoding (manteniéndolo en int8)
    ohe_values = ohe.fit_transform(df[cat_cols]) # OJO: El OHE sí lo dejamos aquí
    clean_ohe_cols = clean_column_names(ohe.get_feature_names_out(cat_cols))
    df_ohe = pd.DataFrame(ohe_values, columns=clean_ohe_cols, index=df.index).astype('int8')
    
    cols_to_keep = ['unique_id', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits']
    return pd.concat([df[cols_to_keep], df_ohe], axis=1).drop_duplicates(subset=['unique_id']).set_index('unique_id')

X_train_demo = process_demographics_raw(train_students)

print(f"✅ Procesado correctamente. Shape: {X_train_demo.shape}")
X_train_demo.head()

✅ Procesado correctamente. Shape: (22785, 26)


,imd_band,age_band,num_of_prev_attempts,studied_credits,gender_f,gender_m,region_east_anglian_region,region_east_midlands_region,region_ireland,region_london_region,...,region_wales,region_west_midlands_region,region_yorkshire_region,highest_education_a_level_or_equivalent,highest_education_he_qualification,highest_education_lower_than_a_level,highest_education_no_formal_quals,highest_education_post_graduate_qualification,disability_n,disability_y
unique_id,,,,,,,,,,,,,,,,,,,,,
11391_AAA_2013J,9,2,0,240,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
28400_AAA_2013J,2,1,0,60,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
32885_AAA_2013J,5,0,0,60,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
45462_AAA_2013J,3,0,0,60,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
45642_AAA_2013J,9,0,0,120,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
import numpy as np
import pandas as pd

def process_interactions(df_interactions):
    print("- Procesando Interacciones (Desglose + Engagement Score + Contexto)...")
    
    # 1. Pivotar: Convertir tipos de actividad en columnas
    df_clicks = df_interactions.groupby(['unique_id', 'activity_type'])['sum_click'].sum().unstack(fill_value=0)
    
    # Limpiar nombres de columnas
    df_clicks.columns = [f'clicks_{col.lower()}' for col in df_clicks.columns]
    
    # 2. Calcular el Engagement Score Ponderado
    weights = {
        'clicks_quiz': 3.0, 
        'clicks_subpage': 2.0, 
        'clicks_oucontent': 2.0,
        'clicks_forumng': 1.5, 
        'clicks_resource': 1.5, 
        'clicks_homepage': 1.0, 
        'clicks_url': 1.0
    }
    
    weighted_score = pd.Series(0, index=df_clicks.index)
    for col, weight in weights.items():
        if col in df_clicks.columns:
            weighted_score += df_clicks[col] * weight
    
    # 3. Métricas agregadas (Absolutas)
    df_clicks['total_weighted_engagement'] = weighted_score
    df_clicks['total_raw_clicks'] = df_clicks.filter(like='clicks_').sum(axis=1)
    df_clicks['activity_diversity'] = (df_clicks.filter(like='clicks_') > 0).sum(axis=1)
    
    # --- NUEVO: CÁLCULO RELATIVO (Contextual por Módulo) ---
    # Objetivo: ¿Este alumno estudia más o menos que la media DE SU CLASE?
    
    # A. Extraemos el contexto (Modulo_Presentacion) del índice 'unique_id'
    # Asumimos formato: '11391_AAA_2013J' -> Split por '_' -> [ID, Modulo, Presentacion]
    index_parts = df_clicks.index.to_series().str.split('_', expand=True)
    
    # Creamos una columna temporal para agrupar (ej. 'AAA_2013J')
    # Usamos las partes 1 y 2 del split (ajusta si tu ID tiene otro formato)
    df_clicks['temp_context_group'] = index_parts[1] + '_' + index_parts[2]
    
    # B. Calculamos Z-Score dentro de cada grupo
    # (Mi Nota - Media del Grupo) / Desviación del Grupo
    grp = df_clicks.groupby('temp_context_group')['total_weighted_engagement']
    context_mean = grp.transform('mean')
    context_std = grp.transform('std').replace(0, 1) # Evitamos división por cero si solo hay 1 alumno
    
    df_clicks['relative_engagement_zscore'] = (df_clicks['total_weighted_engagement'] - context_mean) / context_mean
    
    # C. Limpieza
    df_clicks.drop(columns=['temp_context_group'], inplace=True)
    
    # 4. Log-Scaling (Crítico: NO aplicar al Z-Score que tiene negativos)
    # Filtramos columnas que sean clicks o engagement, PERO excluimos el zscore
    cols_to_log = [c for c in df_clicks.columns 
                   if ('clicks' in c or 'total_weighted' in c) 
                   and 'zscore' not in c]
    
    df_clicks[cols_to_log] = np.log1p(df_clicks[cols_to_log])
    
    return df_clicks

# --- PRUEBA DEL CÓDIGO ---
# (Asegúrate de tener train_interactions cargado)
X_train_inter = process_interactions(train_interactions)

print(f"✅ Variables creadas: {X_train_inter.shape[1]}")
print("✅ Muestra de Engagement Relativo (Z-Score):")
print(X_train_inter[['total_weighted_engagement', 'relative_engagement_zscore']].head())

- Procesando Interacciones con Contexto Relativo...


,clicks_dataplus,clicks_dualpane,clicks_externalquiz,clicks_folder,clicks_forumng,clicks_glossary,clicks_homepage,clicks_htmlactivity,clicks_oucollaborate,clicks_oucontent,...,clicks_repeatactivity,clicks_resource,clicks_sharedsubpage,clicks_subpage,clicks_url,total_weighted_engagement,total_raw_clicks,activity_diversity,rel_eng_zscore,rel_eng_pct
unique_id,,,,,,,,,,,,,,,,,,,,,
100064_FFF_2013J,3.295837,2.833213,0.000000,1.94591,6.342122,0.0,6.677083,0.0,1.386294,8.177516,...,0.0,4.234107,0.0,6.381816,4.158883,9.411034,8.781862,14.0,1.683527,2.534306
100561_DDD_2014J,0.000000,0.000000,3.496508,0.00000,5.805135,0.0,5.407172,0.0,2.484907,4.488636,...,0.0,4.343805,0.0,5.068904,3.496508,7.213400,6.857514,8.0,0.064933,1.090876
100621_CCC_2014B,0.000000,0.000000,0.000000,0.00000,0.693147,0.0,3.433987,0.0,0.000000,2.197225,...,0.0,1.098612,0.0,2.833213,0.693147,4.471639,4.094345,7.0,-0.709605,0.034414
1006742_FFF_2014B,0.000000,0.693147,0.000000,0.00000,5.857933,0.0,6.267200,0.0,1.098612,6.625392,...,0.0,4.543295,0.0,5.739793,3.850147,8.280711,7.792349,12.0,-0.097644,0.907907
1008675_BBB_2013B,0.000000,0.000000,0.000000,0.00000,6.928538,0.0,6.371612,0.0,0.000000,0.000000,...,0.0,4.110874,0.0,4.248495,2.995732,7.950150,7.556951,7.0,0.698022,2.096789


In [19]:
import numpy as np
import pandas as pd
from scipy.stats import linregress

def process_performance_v2(df_assessments):
    print("- Procesando Rendimiento (Robust Trend + Punctuality)...")
    
    # 1. Limpieza y ORDENAMIENTO (Crítico para Trend)
    # Ordenamos por alumno y fecha límite para que la secuencia temporal sea real
    df_assess = df_assessments[df_assessments['assessment_type'] != 'Exam'].copy()
    df_assess.sort_values(by=['unique_id', 'date'], inplace=True)
    
    # Conversiones
    cols_to_num = ['score', 'date_submitted', 'date', 'weight']
    for col in cols_to_num:
        df_assess[col] = pd.to_numeric(df_assess[col], errors='coerce').fillna(0)
    
    # 2. Ingeniería de Características
    
    # A. Weighted Score (Puntos reales obtenidos)
    # Nota: Mantenemos tu lógica, asumiendo que el modelo aprenderá el contexto del curso
    df_assess['weighted_score'] = df_assess['score'] * (df_assess['weight'] / 100.0)
    
    # B. Submit Delay (Con penalización por no entrega implícita en count)
    df_assess['submission_delay'] = df_assess['date_submitted'] - df_assess['date']
    df_assess['is_late'] = (df_assess['submission_delay'] > 0).astype(int)
    
    # C. Pass Flag (¿Aprobó esta tarea?) - OULAD suele usar 40 como corte
    df_assess['is_passed'] = (df_assess['score'] >= 40).astype(int)

    # 3. Función de Tendencia Robusta (Slope)
    # Usamos pendiente de regresión lineal. Es más robusta que "mitad vs mitad"
    # Si la pendiente es positiva, el alumno mejora. Si es negativa, empeora.
    def calculate_slope(x):
        if len(x) < 2: return 0.0
        # Eje X: índice de entrega (0, 1, 2...), Eje Y: notas
        slope, _, _, _, _ = linregress(range(len(x)), x)
        return slope if not np.isnan(slope) else 0.0

    # 4. Agregación (Optimizada)
    # Agrupamos una sola vez para eficiencia
    grouped = df_assess.groupby('unique_id')
    
    perf_df = grouped.agg({
        'score': ['mean', 'std', calculate_slope], # Slope es mejor que split-half
        'weighted_score': 'sum',
        'is_late': ['sum', 'mean'], # 'mean' nos da el ratio de entregas tarde directo
        'submission_delay': 'mean',
        'is_passed': 'mean', # Ratio de aprobados (ej. 0.8 = aprobó el 80% de entregas)
        'id_assessment': 'count'
    })
    
    # Aplanar columnas MultiIndex
    perf_df.columns = [
        'avg_score', 'score_std', 'score_slope', 
        'api_index', 
        'late_count', 'late_ratio', 
        'avg_delay', 
        'pass_ratio', 
        'submission_count'
    ]
    
    # 5. Tratamiento de Nulos (Post-Agregación)
    # std puede ser NaN si solo hay 1 entrega. Rellenamos con 0.
    perf_df.fillna(0, inplace=True)
    
    # 6. Optimización de memoria
    return perf_df.astype('float32')

# Ejecutar
X_train_perf = process_performance_v2(train_assessments)
X_train_perf.head()

- Procesando Rendimiento (Robust Trend + Punctuality)...


,avg_score,score_std,score_slope,api_index,late_count,late_ratio,avg_delay,pass_ratio,submission_count
unique_id,,,,,,,,,
100064_FFF_2013J,89.333336,4.773665,-0.510490,92.000000,2.0,0.166667,-23.250000,1.0,12.0
100561_DDD_2014J,73.166664,10.759492,-2.542857,72.500000,0.0,0.000000,-0.166667,1.0,6.0
1006742_FFF_2014B,78.500000,0.707107,1.000000,9.750000,0.0,0.000000,-95.000000,1.0,2.0
1008675_BBB_2013B,76.909088,21.421740,4.145454,68.379997,7.0,0.636364,3.545455,1.0,11.0
100893_AAA_2013J,68.400002,4.277850,1.200000,68.699997,2.0,0.400000,0.200000,1.0,5.0


In [21]:
import numpy as np
import pandas as pd
from scipy.stats import linregress

def process_behavioral_advanced(df_interactions):
    print("- Procesando Métricas Avanzadas de Comportamiento (Slope, Weekend, Curiosity)...")
    
    # Trabajamos sobre una copia para no alterar el original
    df = df_interactions.copy()
    
    # ---------------------------------------------------------
    # 1. ACELERACIÓN (LA DERIVADA DEL ESFUERZO)
    # ---------------------------------------------------------
    # Hipótesis: ¿El alumno está aumentando su ritmo (slope > 0) o abandonando (slope < 0)?
    
    # Agrupamos por semana para suavizar el ruido diario
    # OULAD usa días, así que dividimos por 7 para obtener semanas
    df['week'] = np.floor(df['date'] / 7)
    
    # Calculamos la suma de clics por semana para cada alumno
    weekly_clicks = df.groupby(['unique_id', 'week'])['sum_click'].sum().reset_index()
    
    # Función para calcular la pendiente (slope) de la regresión lineal
    def calculate_effort_slope(group):
        if len(group) < 2: return 0.0
        # x = número de semana, y = clics
        slope, _, _, _, _ = linregress(group['week'], group['sum_click'])
        return slope if not np.isnan(slope) else 0.0

    # Aplicamos la función (esto puede tardar unos segundos)
    # Resultado: Un valor positivo indica que estudia cada vez más. Negativo, que se apaga.
    effort_slope = weekly_clicks.groupby('unique_id').apply(calculate_effort_slope)
    effort_slope.name = 'effort_slope'
    
    # ---------------------------------------------------------
    # 2. WEEKEND RATIO (PROCRASTINACIÓN)
    # ---------------------------------------------------------
    # Hipótesis: En OULAD, el día 0 suele ser el inicio de la semana de estudio.
    # Asumimos que los días 5 y 6 (o 6 y 0 mod 7) son el fin de semana.
    # Calculamos qué % de sus clics ocurren en días % 7 >= 5
    
    df['day_of_week'] = df['date'] % 7
    # Definimos fin de semana como los días 5 y 6 del ciclo semanal
    df['is_weekend'] = df['day_of_week'].isin([5, 6])
    
    weekend_stats = df.groupby('unique_id').apply(
        lambda x: x.loc[x['is_weekend'], 'sum_click'].sum() / (x['sum_click'].sum() + 1e-5)
    )
    weekend_stats.name = 'weekend_ratio'
    
    # ---------------------------------------------------------
    # 3. ÍNDICE DE CURIOSIDAD (EXPLORACIÓN VS EXPLOTACIÓN)
    # ---------------------------------------------------------
    # Hipótesis: Alumnos que visitan recursos opcionales tienen mayor motivación intrínseca.
    
    # Definimos actividades "Auxiliares" (Curiosidad) vs "Core" (Obligación)
    auxiliary_activities = ['glossary', 'oucollaborate', 'resource', 'forumng', 'dataplus']
    
    # Filtramos solo actividades auxiliares
    aux_clicks = df[df['activity_type'].isin(auxiliary_activities)].groupby('unique_id')['sum_click'].sum()
    total_clicks = df.groupby('unique_id')['sum_click'].sum()
    
    # Ratio: Clics Auxiliares / Clics Totales
    # Usamos reindex para asegurar que están todos los alumnos (incluso los que tienen 0 aux)
    curiosity_index = (aux_clicks / (total_clicks + 1e-5)).fillna(0)
    curiosity_index.name = 'curiosity_index'
    
    # ---------------------------------------------------------
    # UNIÓN FINAL
    # ---------------------------------------------------------
    advanced_features = pd.concat([effort_slope, weekend_stats, curiosity_index], axis=1).fillna(0)
    
    # Optimización de memoria
    return advanced_features.astype('float32')

# --- EJEMPLO DE USO ---
X_train_adv = process_behavioral_advanced(train_interactions)
print(X_train_adv.head())

- Procesando Métricas Avanzadas de Comportamiento (Slope, Weekend, Curiosity)...
                   effort_slope  weekend_ratio  curiosity_index
unique_id                                                      
100064_FFF_2013J       0.493223       0.269727         0.101934
100561_DDD_2014J      -0.022605       0.198947         0.440000
100621_CCC_2014B      -7.348837       0.745763         0.050847
1006742_FFF_2014B    -20.147806       0.307311         0.183395
1008675_BBB_2013B     -1.167181       0.250392         0.564558
